In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

#define the modified AlexNet
class AlexNetModified(nn.Module):
    def __init__(self, num_classes=10, use_dropout=False):
        super(AlexNetModified, self).__init__()
        self.use_dropout = use_dropout
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5) if use_dropout else nn.Identity(),

            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5) if use_dropout else nn.Identity(),

            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

##############Function to count model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#############Load CIFAR-10 & CIFAR-100 datasets
trainloader_10, testloader_10, trainloader_100, testloader_100 = None, None, None, None

def get_data_loaders(batch_size=128):
    global trainloader_10, testloader_10, trainloader_100, testloader_100
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset_10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset_10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    trainset_100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    testset_100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
    trainloader_10, testloader_10 = DataLoader(trainset_10, batch_size=batch_size, shuffle=True), DataLoader(testset_10, batch_size=batch_size, shuffle=False)
    trainloader_100, testloader_100 = DataLoader(trainset_100, batch_size=batch_size, shuffle=True), DataLoader(testset_100, batch_size=batch_size, shuffle=False)

#######Initialize data loaders
get_data_loaders()

#####Train function
def train_model(model, trainloader, testloader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(trainloader))

        model.eval()
        correct, total, val_loss = 0, 0, 0.0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_losses.append(val_loss / len(testloader))
        val_accuracies.append(correct / total * 100)
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.2f}%")

    return train_losses, val_losses, val_accuracies

######Train CIFAR-10 models
print("Training AlexNet Without Dropout on CIFAR-10")
train_losses_no_dropout_10, val_losses_no_dropout_10, val_acc_no_dropout_10 = train_model(AlexNetModified(num_classes=10, use_dropout=False), trainloader_10, testloader_10)
print("Training AlexNet With Dropout on CIFAR-10")
train_losses_dropout_10, val_losses_dropout_10, val_acc_dropout_10 = train_model(AlexNetModified(num_classes=10, use_dropout=True), trainloader_10, testloader_10)

######Train CIFAR-100 models
print("Training AlexNet Without Dropout on CIFAR-100")
train_losses_no_dropout_100, val_losses_no_dropout_100, val_acc_no_dropout_100 = train_model(AlexNetModified(num_classes=100, use_dropout=False), trainloader_100, testloader_100)
print("Training AlexNet With Dropout on CIFAR-100")
train_losses_dropout_100, val_losses_dropout_100, val_acc_dropout_100 = train_model(AlexNetModified(num_classes=100, use_dropout=True), trainloader_100, testloader_100)

### Results table
results = pd.DataFrame({
    "Dataset": ["CIFAR-10", "CIFAR-10", "CIFAR-100", "CIFAR-100"],
    "Dropout": ["No", "Yes", "No", "Yes"],
    "Final Val Accuracy (%)": [val_acc_no_dropout_10[-1], val_acc_dropout_10[-1], val_acc_no_dropout_100[-1], val_acc_dropout_100[-1]],
    "Final Val Loss": [val_losses_no_dropout_10[-1], val_losses_dropout_10[-1], val_losses_no_dropout_100[-1], val_losses_dropout_100[-1]]
})
print(results)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Training AlexNet Without Dropout on CIFAR-10
Epoch 1/10 - Train Loss: 1.6413, Val Loss: 1.3055, Val Acc: 51.94%
Epoch 2/10 - Train Loss: 1.1862, Val Loss: 1.0719, Val Acc: 61.84%
Epoch 3/10 - Train Loss: 0.9689, Val Loss: 0.8917, Val Acc: 68.43%
Epoch 4/10 - Train Loss: 0.8328, Val Loss: 0.8472, Val Acc: 70.54%
Epoch 5/10 - Train Loss: 0.7378, Val Loss: 0.7374, Val Acc: 74.98%
Epoch 6/10 - Train Loss: 0.6713, Val Loss: 0.7071, Val Acc: 75.51%
Epoch 7/10 - Train Loss: 0.6151, Val Loss: 0.6742, Val Acc: 76.75%
Epoch 8/10 - Train Loss: 0.5691, Val Loss: 0.6194, Val Acc: 78.18%
Epoch 9/10 - Train Loss: 0.5324, Val Loss: 0.6050, Val Acc: 79.50%
Epoch 10/10 - Train Loss: 0.5026, Val Loss: 0.6089, Val Acc: 79.52%
Training AlexNet With Dropout on CIFAR-10
Epoch 1/10 - Train Loss: 1.7561, Val Loss: 1.4997, Val Acc: 44.55%
Epoch 2/10 - Train Loss